In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Multi genbank and stacking



In [ ]:
#Using the example E. coli genome data from the package
import genomenotebook as gn
from genomenotebook.utils import get_cds_name, add_z_order, parse_genbank
import os
import pandas as pd
from Bio import SeqIO
from bokeh.io import output_notebook #|hide_line
from bokeh.plotting import show as bk_show #|hide_line
from bokeh.layouts import column, row #|hide_line
from bokeh.plotting import output_file, save #|hide_line
from collections import OrderedDict

In [ ]:

def plot_multi_genbank(gb_path, init_pos=None, feature_types=["CDS", "repeat_region", "ncRNA", "rRNA", "tRNA"], name_func=get_cds_name, show_labels=True):
    """
        recs: iterator of SeqRecords
        init_pos: where to center the plot
    """
    seq_ids=[rec.id for rec in SeqIO.parse(gb_path,"genbank")]
    names=seq_ids
    browsers = []
    for seq_id in seq_ids:
        browsers.append(gn.GenomeBrowser(gb_path=gb_path, 
                                         seq_id=seq_id,
                                         init_pos=init_pos,
                                         width=1000, 
                                         show_seq=False, 
                                         search=False, 
                                         feature_types=feature_types, 
                                         color_attribute="Color", 
                                         label_angle=0, 
                                         show_labels=show_labels, 
                                         feature_height=0.15, 
                                         label_vertical_offset=-0.15, 
                                         label_justify="left", 
                                         label_horizontal_offset = 5, 
                                         glyphs=gn.get_default_glyphs(arrow_colors=("Cyan",), box_colors=("Cyan",)), 
                                         feature_name={"CDS":"gene_id","Domainator":"name"},
                                         z_stack=True
                                         )
                        )

        browsers[-1]._get_browser_elements()

    browsers[0].gene_track.xaxis.axis_label = names[0]
    if len(browsers) > 1:
        browsers[0].gene_track.xaxis.major_tick_line_color = None
        browsers[0].gene_track.xaxis.minor_tick_line_color = None
        browsers[0].gene_track.xaxis.major_label_text_font_size  = '0pt'

    for i, browser in enumerate(browsers[1:]):
        i = i+1
        track = browsers[0].add_track()
        track.fig = browsers[i].gene_track
        track.fig.axis.axis_label = names[i]
        track.fig.x_range = browsers[0].x_range
        if i < len(browsers)-1:
            track.fig.xaxis.major_tick_line_color = None
            track.fig.xaxis.minor_tick_line_color = None
            track.fig.xaxis.major_label_text_font_size  = '0pt'

    return browsers[0]


output_notebook(hide_banner=True) #|hide_line
data_path = gn.get_example_data_dir()
gb_path = os.path.join(data_path, "colored_genbank.gb")
g = plot_multi_genbank(gb_path, feature_types=["CDS", "Domainator"], show_labels=False) #, "Domainator", "Domain_Search"
g.show()
#g.save_html("test2.html", title="stacked_contigs")

In [ ]:
g.show()